## Import

In [34]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import jinja2
import sklearn
from sklearn.preprocessing import StandardScaler

# Add the project root directory to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Added '{project_root}' to sys.path")

import src.load_libs 
from src.load_data import load_stroke_data 


## Loading data

In [35]:
print("Loading data...")
df = load_stroke_data()
display(df.head())

Loading data...
Attempting to load file: healthcare-dataset-stroke-data.csv from dataset fedesoriano/stroke-prediction-dataset
Dataset loaded successfully!


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


## Encoding data

ML algos need numerical features, so we need to convert the categorical features to numerical features.

In [36]:
if df is not None:
    print("DataFrame shape before encoding:", df.shape)
    print("Columns before encoding:", df.columns.tolist())
    print("\nData types before encoding:")
    df.info()
    df.head()

    # Identify categorical columns to encode (excluding binary 0/1 already numerical)
    # We'll encode 'object' type columns. We can also explicitly list them.
    categorical_to_encode = df.select_dtypes(include=['object']).columns.tolist()
    # Or be explicit:
    # categorical_to_encode = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
    categorical_to_encode = [col for col in categorical_to_encode if col in df.columns]

    print(f"\nCategorical columns to one-hot encode: {categorical_to_encode}")

    if categorical_to_encode:
        # Use pandas get_dummies for one-hot encoding
        # drop_first=True can help reduce multicollinearity by dropping one category per feature
        df_encoded = pd.get_dummies(df, columns=categorical_to_encode, drop_first=True, dtype=int)

        print("\nDataFrame shape after encoding:", df_encoded.shape)
        print("Columns after encoding:", df_encoded.columns.tolist())

        # Display the first few rows with new columns
        print("\nFirst 5 rows after encoding:")
        display(df_encoded.head())

        # Update df to the encoded version
        df = df_encoded
        print("\nDataFrame 'df' updated with encoded categorical variables.")
    else:
        print("\nNo categorical columns found or specified for encoding.")

else:
    print("DataFrame not loaded.")

DataFrame shape before encoding: (5110, 12)
Columns before encoding: ['id', 'gender', 'age', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'avg_glucose_level', 'bmi', 'smoking_status', 'stroke']

Data types before encoding:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,9046,67.0,0,1,228.69,36.6,1,1,0,1,0,1,0,0,1,1,0,0
1,51676,61.0,0,0,202.21,NaN,1,0,0,1,0,0,1,0,0,0,1,0
2,31112,80.0,0,1,105.92,32.5,1,1,0,1,0,1,0,0,0,0,1,0
3,60182,49.0,0,0,171.23,34.4,1,0,0,1,0,1,0,0,1,0,0,1
4,1665,79.0,1,0,174.12,24.0,1,0,0,1,0,0,1,0,0,0,1,0



DataFrame 'df' updated with encoded categorical variables.


## Scaling

Now we have to scale our features, ML perform better when numerical features are scaled.  
For example, age, avg_glucole_lvl, bmi ae on very different ranges, so we need to scale them.

To do this, we will use standardization.

In [37]:
from sklearn.preprocessing import StandardScaler

if 'df' in locals() and df is not None:
    # Identify the numerical columns that need scaling
    # These are the original continuous numerical cols, not the binary/encoded ones or the target/id
    cols_to_scale = [col for col in ['age', 'avg_glucose_level', 'bmi'] if col in df.columns]

    if cols_to_scale:
        print(f"Columns to scale: {cols_to_scale}")

        # Create a copy to avoid modifying the original DataFrame slice directly
        df_processed = df.copy()

        # Initialize the scaler
        scaler = StandardScaler()

        # Fit the scaler to the data and transform the columns
        df_processed[cols_to_scale] = scaler.fit_transform(df_processed[cols_to_scale])

        print("\nDataFrame shape after scaling:", df_processed.shape)
        print("Columns after scaling:", df_processed.columns.tolist())
        print("\nFirst 5 rows after scaling:")
        display(df_processed.head())
        print("\nDescriptive stats for scaled columns:")
        # Mean should be close to 0, std dev close to 1
        display(df_processed[cols_to_scale].describe())

        # Update df to the processed version
        df = df_processed
        print("\nDataFrame 'df' updated with scaled numerical features.")

    else:
        print("\nNo columns identified or available for scaling.")

else:
    print("DataFrame 'df' not found or not loaded.")


Columns to scale: ['age', 'avg_glucose_level', 'bmi']

DataFrame shape after scaling: (5110, 18)
Columns after scaling: ['id', 'age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi', 'stroke', 'gender_Male', 'gender_Other', 'ever_married_Yes', 'work_type_Never_worked', 'work_type_Private', 'work_type_Self-employed', 'work_type_children', 'Residence_type_Urban', 'smoking_status_formerly smoked', 'smoking_status_never smoked', 'smoking_status_smokes']

First 5 rows after scaling:


,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,gender_Other,ever_married_Yes,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,9046,1.051434,0,1,2.706375,0.981345,1,1,0,1,0,1,0,0,1,1,0,0
1,51676,0.786070,0,0,2.121559,NaN,1,0,0,1,0,0,1,0,0,0,1,0
2,31112,1.626390,0,1,-0.005028,0.459269,1,1,0,1,0,1,0,0,0,0,1,0
3,60182,0.255342,0,0,1.437358,0.701207,1,0,0,1,0,1,0,0,1,0,0,1
4,1665,1.582163,1,0,1.501184,-0.623083,1,0,0,1,0,0,1,0,0,0,1,0



Descriptive stats for scaled columns:


,age,avg_glucose_level,bmi
count,5.110000e+03,5.110000e+03,4.909000e+03
mean,5.005781e-17,1.001156e-16,-4.487029e-16
std,1.000098e+00,1.000098e+00,1.000102e+00
min,-1.908261e+00,-1.126958e+00,-2.367580e+00
25%,-8.061152e-01,-6.383223e-01,-6.867508e-01
50%,7.843218e-02,-3.149945e-01,-1.010073e-01
75%,7.860701e-01,1.754080e-01,5.356705e-01
max,1.714845e+00,3.657145e+00,8.748813e+00



DataFrame 'df' updated with scaled numerical features.
